<a href="https://colab.research.google.com/github/RaiGon84/Pytorch-Tutorial/blob/main/MNIST_tutorial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [46]:
# Download dataset
def download_mnist_datasets():
  train_data = datasets.MNIST(
      root="data",
      download=True,
      train=True,
      transform=ToTensor()
  )
  validation_data = datasets.MNIST(
      root="data",
      download=True,
      train=False,
      transform=ToTensor()
  )
  return train_data, validation_data

if __name__ == "__main__":
  train_data, _ = download_mnist_datasets()
  print(f"MNIST dataset downloaded")




MNIST dataset downloaded


In [47]:
# Create a data loader for the training set
BATCH_SIZE = 128
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)


In [48]:
class FeedForwardNet(nn.Module):

  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.dense_layers = nn.Sequential(
        nn.Linear(28*28, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input_data):
    flattened_data = self.flatten(input_data)
    logits = self.dense_layers(flattened_data)
    predictions = self.softmax(logits)
    return predictions

In [49]:
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(f"Using {device}")

Using cuda


In [50]:
# Build model
feed_forward_net = FeedForwardNet().to(device)

In [51]:
EPOCHS = 10
def train_single_epoch(model, data_loader,loss_fn,optimizer,device):
  for inputs, targets in data_loader:
    inputs, targets = inputs.to(device), targets.to(device)

    # calculate loos
    predictions = model(inputs)
    loss = loss_fn(predictions, targets)

    # backpropagate loss and update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"loss: {loss.item()}")

def train(model,data_loader,loss_fn,optimizer,device,epochs):
  for i in range(epochs):
    print(f"Epoch {i+1}")
    train_single_epoch(model, data_loader, loss_fn, optimizer,device)
    print("---------------------------")
  print("Finished training")

In [52]:
# Instatiate loss function
loss_fn = nn.CrossEntropyLoss()
# Instatiate optimizer
optimizer = torch.optim.Adam(feed_forward_net.parameters(), lr=0.001)

# Train model
train(feed_forward_net,train_dataloader,loss_fn,optimizer,device,EPOCHS)

Epoch 1
loss: 1.5425134897232056
---------------------------
Epoch 2
loss: 1.5300847291946411
---------------------------
Epoch 3
loss: 1.5178064107894897
---------------------------
Epoch 4
loss: 1.5102125406265259
---------------------------
Epoch 5
loss: 1.4801727533340454
---------------------------
Epoch 6
loss: 1.4826531410217285
---------------------------
Epoch 7
loss: 1.4926677942276
---------------------------
Epoch 8
loss: 1.5118821859359741
---------------------------
Epoch 9
loss: 1.4624303579330444
---------------------------
Epoch 10
loss: 1.4641543626785278
---------------------------
Finished training


In [53]:
torch.save(feed_forward_net.state_dict(), "feedforwardnet.pth")
print(f"Model train and stored as feedforwardnet.pth")

Model train and stored as feedforwardnet.pth
